# Popularity Prediction for UCSD Courses

In [ ]:
from popularity import * # TODO: fix imports
# TODO: I think would be smart to use the runner code from popularity.py within this file so we can work on migrating that stuff here
from baseline import * # TODO: fix imports
import random

## helper functions

In [ ]:
def clip(x, min_val, max_val):
    return max(min_val, min(x, max_val))

## read in data

## baseline runner

In [ ]:
"""
baseline rating prediction (from baselines.py)
"""
allRatings = []
userRatings = defaultdict(list)

for user, book, rating in ratingsTrain:
    allRatings.append(rating)
    userRatings[user].append(rating)

globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
for u in userRatings:
    userAverage[u] = sum(userRatings[u]) / len(userRatings[u])

# compute mse on validation set
mse = 0
for user, book, rating in ratingsValid:
    if user in userAverage:
        pred = userAverage[user]
    else:
        pred = globalAverage
    mse += (pred - rating) ** 2
mse /= len(ratingsValid)

## runner code for popularity prediction

In [ ]:
"""
train and evaluate rating prediction model
"""
# hyperparameters
lambdaU = 3.5
lambdaI = 16
validationSplit = 0.1
finalTrainingRounds = 10

# file paths
trainPath = "" # TODO: fix path

# load all ratings
allRatings = []
for user, book, rating in readCSV(trainPath):
    allRatings.append((user, book, rating))

# split into train/validation
random.seed(42)
random.shuffle(allRatings)
split_idx = int(len(allRatings) * (1 - validationSplit))
ratingsTrain = allRatings[:split_idx]
ratingsValid = allRatings[split_idx:]

# build per-user and per-item dictionaries
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)

for user, item, rating in ratingsTrain:
    ratingsPerUser[user].append((item, rating))
    ratingsPerItem[item].append((user, rating))

print(
    f"\ntraining on {len(ratingsTrain)} ratings, validating on {len(ratingsValid)} ratings"
)
print(f"unique users: {len(ratingsPerUser)}, unique items: {len(ratingsPerItem)}")

# baseline model
print("\n" + "=" * 60)
print("BASELINE MODEL")
print("=" * 60)
baseline_MSE = runRatingBaselinePrediction(ratingsTrain, ratingsValid)
print(f"baseline validation MSE: {baseline_MSE:.4f}")

# train model with validation
print("\n" + "=" * 60)
print("MY MODEL (Bias-Only Model)")
print(f"lambdaU: {lambdaU}, lambdaI: {lambdaI}")
print("=" * 60)

alpha, betaU, betaI, my_MSE = myRatingModel(
    ratingsTrain,
    ratingsValid,
    ratingsPerUser,
    ratingsPerItem,
    lambdaU,
    lambdaI,
)

print(f"\nfinal validation MSE: {my_MSE:.4f}")

print("\n" + "=" * 60)
print("PERFORMANCE COMPARISON")
print("=" * 60)
print(f"baseline MSE: {baseline_MSE:.4f}")
print(f"my MSE:       {my_MSE:.4f}")
improvement = baseline_MSE - my_MSE
improvement_pct = (improvement / baseline_MSE) * 100
print(f"improvement:  {improvement:+.4f} ({improvement_pct:+.2f}%)")

# retrain on all data for final rounds
print("\n" + "=" * 60)
print(f"FINAL TRAINING: {finalTrainingRounds} rounds on all data")
print("=" * 60)

# rebuild per-user and per-item dictionaries with all data
ratingsPerUserAll = defaultdict(list)
ratingsPerItemAll = defaultdict(list)

for user, item, rating in allRatings:
    ratingsPerUserAll[user].append((item, rating))
    ratingsPerItemAll[item].append((user, rating))

# continue training from the current parameters
for i in range(finalTrainingRounds):
    alpha = alphaUpdate(allRatings, alpha, betaU, betaI)
    betaU = betaUUpdate(ratingsPerUserAll, alpha, betaU, betaI, lambdaU)
    betaI = betaIUpdate(ratingsPerItemAll, alpha, betaU, betaI, lambdaI)

    trainMSE, trainMSEReg = msePlusReg(
        allRatings,
        alpha,
        betaU,
        betaI,
        lambdaU,
        lambdaI,
    )

    print(
        f"Round {i + 1}: Training MSE = {trainMSE:.4f}, MSE+Reg = {trainMSEReg:.4f}"
    )

print(f"\nfinal training MSE after combined data: {trainMSE:.4f}")

my_MSE, baseline_MSE
